##### 1. 필요한 라이브러리 및 패키지 임포트

In [1]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import time
import re
import random

##### 2. url 생성 함수

In [13]:
def makePgNum(num): # 페이지 넘버링 함수
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)


# 크롤링할 url 생성 함수
def makeUrl(search,start_pg,end_pg):
    if start_pg == end_pg:
        page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&sort=1&start=" + str(page)
        return url
    else:
        urls= []
        for i in range(start_pg,end_pg+1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&sort=1&start=" + str(page)
            urls.append(url)
        return urls # 전체 url 리턴

In [14]:
# 네이버 뉴스 링크 추출 함수
def get_naverlink(search_urls):

    naverlink_list=[]
    
    for i in search_urls:
        r=requests.get(i) # requests.get(url)로 요청
        time.sleep(random.randint(1,3)) # 요청시간 랜덤 조정으로 크롤링 차단 방지
    
        soup=BeautifulSoup(r.text, "html.parser") # html.parser로 파싱
        content_items= soup.select(".info_group") # select로 태그 선택
        content_items=str(content_items) # 문자열 변환
    
        pattern = r'href="([^"]+)"'
        links = re.findall(pattern, content_items) # 링크만 추출
    
        naver_word='news.naver.com' 
    
        for j in links:
            if naver_word in j:
                naverlink_list.append(j)
    
    return naverlink_list # 전체 네이버 뉴스 링크 리턴

##### 3. 뉴스 기사 크롤러

In [15]:
# naver 기사 제목, 본문, 시간 가져오기

def get_title_content(naverlink_list):
    
    # ConnectionError방지
    headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
            "Accept-Language": "ko-KR,ko;q=0.9",
            "Accept-Encoding": "gzip, deflate, br",
            "Connection": "keep-alive",
            "Upgrade-Insecure-Requests": "1",
            "TE": "Trailers",
            "DNT": "1",
        }
    
    contents=[] # 본문
    titles=[] # 제목
    times=[] # 작성 시간

    for i in naverlink_list:
        r = requests.get(i,headers=headers)
        time.sleep(random.radint(1,3)) # 요청시간 랜덤 조정으로 크롤링 차단 방지
        soup = BeautifulSoup(r.text, "html.parser")

        content = soup.find('div',{'class': 'newsct_article _article_body'})
        title =soup.find('div',{'class': 'media_end_head_title'})
        time=soup.find('div',{'class': 'media_end_head_info_datestamp_bunch'})
    
        #제목 텍스트만 가져오기
        title=title.text
        title=title.replace("\n","")
        titles.append(title) 
         
        # 기사 텍스트만 가져오기
        content=content.text.replace("\n","")
        contents.append(content)
        
        # 시간 정보 텍스트만
        time=time.text
        time=time.replace("\n","")
        time=time.replace("입력","")
        time=time[:11]
        times.append(time)
        
    return contents, titles, times

##### 4. url 생성 함수 호출

In [39]:
search = input("검색할 키워드를 입력해주세요:")

#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요.:"))
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요.:"))

# naver url 생성
search_urls = makeUrl(search,page,page2)

# naver news url 추출
naverlink_list = get_naverlink(search_urls) # 네이버 뉴스 링크 생성

검색할 키워드를 입력해주세요: 반려견 건식사료 영향

크롤링할 시작 페이지를 입력해주세요.: 1

크롤링할 종료 페이지를 입력해주세요.: 50


##### 5. 크롤링 작동

In [27]:
naverlink_list = set(naverlink_list) # 뉴스 링크 중복 제거
naverlink_list = list(naverlink_list) # 다시 리스트 형태로 변환

contents, titles, times = get_title_content(naverlink_list) # 크롤러 호출
news_df = pd.DataFrame({'Title':titles,'Time': times,'Link':naverlink_list,'Content':contents}) # 데이터프레임으로 크롤링 데이터 구조화

print(len(news_df)) 

73


##### 6. 데이터 저장

- 여러 검색 키워드로 크롤링 했을 경우 실행

In [ ]:
# news_df=pd.concat([news_dfn, news_dfn, news_dfn])
# news_df=news_df.drop_duplicates()
# print(len(news_df))

news_df.to_csv('NaverNews_content.csv', index=False, encoding='utf-8-sig') # 한국어 저장이 안정적인 인코딩 방식